In [1]:
import numpy as np
import pandas as pd

In [2]:
#define nodes for tree
class Node:
    def __init__(self, theta, index, value=None):
        self.theta = theta
        self.index = index
        self.value = value
        self.leftNode = None
        self.rightNode = None

In [3]:
# Gini index
def gini(Y):
    l = Y.shape[0]
    if l == 0:
        return 1
    return 1-(float(np.sum(Y==1))/l)**2-(float(np.sum(Y==-1))/l)**2

In [4]:
# 为了便于实现，找出每一维度下的最佳划分阈值和对应的branch值 --- 但这样实现代价是运行速度
# 单维情况下的最佳树桩---大于等于为1类
def one_stump(X, Y, thres):
    l = thres.shape[0]
    mini = Y.shape[0]
    for i in range(l):
        Y1 = Y[X<thres[i]]
        Y2 = Y[X>=thres[i]]
        judge = Y1.shape[0]*gini(Y1)+Y2.shape[0]*gini(Y2)
        if mini>judge:
            mini = judge; b = thres[i]
    return mini, b

In [5]:
# 找出最佳划分的阈值和对应的维度
# 结合全部维数的决策树桩
def decision_stump(X, Y):
    row, col = X.shape
    Xsort = np.sort(X, 0)
    thres = (np.r_[Xsort[0:1, :]-0.1, Xsort]+np.r_[Xsort, Xsort[-1:,:]+0.1])/2
    mpurity = row; mb = 0; index = 0
    for i in range(col):
        purity, b = one_stump(X[:,i], Y[:, 0], thres[:, i])
        if mpurity > purity:
            mpurity = purity; mb = b; index =i
    return mb, index

In [6]:
# define stop condition
def stop_cond(X, Y):
    if np.sum(Y!=Y[0]) == 0 or X.shape[0] == 1 or np.sum(X!=X[0, :]) == 0:
        return True
    return False

In [7]:
# fully grown trees
def dTree(X, Y):
    if stop_cond(X, Y):
        node = Node(None, None, Y[0])
        return node
    b, index = decision_stump(X, Y)
    pos1 = X[:, index] < b; pos2 = X[:, index] >= b
    leftX = X[pos1, :]; leftY = Y[pos1, 0:1]
    rightX = X[pos2, :]; rightY = Y[pos2, 0:1]
    node = Node(b, index) # node contains value(stopped); dimension and index(unstopped)
    node.leftNode = dTree(leftX, leftY)
    node.rightNode = dTree(rightX, rightY)
    return node

In [8]:
# Tree based on one decision. Extreme prune
def dTree_one(X, Y):
    b, index = decision_stump(X, Y)
    pos1 = X[:, index] < b; pos2 = X[:, index] >= b
    node = Node(b, index)
    value1 = 1 if np.sign(np.sum(Y[pos1])) >=0 else -1
    value2 = 1 if np.sign(np.sum(Y[pos2])) >=0 else -1
    node.leftNode = Node(None, None, np.array([value1]))
    node.rightNode = Node(None, None, np.array([value2]))
    return node

In [9]:
# prediction on only ONE sample
def predict_one(node, X):
    if node.value is not None:
        return node.value[0]
    thre = node.theta; index = node.index
    if X[index] < thre:
        return predict_one(node.leftNode, X)
    else:
        return predict_one(node.rightNode, X)

In [10]:
# error function
def err_fun(X, Y, node):
    row, col = X.shape
    Yhat = np.zeros(Y.shape)
    for i in range(row):
        Yhat[i] = predict_one(node, X[i, :])
    return Yhat, float(np.sum(Yhat!=Y))/row

In [11]:
# bagging
def bagging(X, Y):
    row, col = X.shape
    pos = np.random.randint(0, row, (row, ))
    return X[pos, :], Y[pos, :]

In [12]:
# random forest on samples
def random_forest(X, Y, T):
    nodeArr = []
    for i in range(T):
        Xtemp, Ytemp = bagging(X, Y)
        node = dTree(Xtemp, Ytemp)
        nodeArr.append(node)
    return nodeArr

In [13]:
# pruned random forest
def random_forest_pruned(X, Y, T):
    nodeArr = []
    for i in range(T):
        Xtemp, Ytemp = bagging(X, Y)
        node = dTree_one(Xtemp, Ytemp)
        nodeArr.append(node)
    return nodeArr

In [14]:
# ----------------具体题目-------------------
# loaddata
def loaddata(filename):
    data = pd.read_csv(filename, sep='\s+', header=None)
    data = data.as_matrix()
    row, col = data.shape
    X = data[:, 0:col-1]
    Y = data[:, col-1:col]
    return X, Y

In [15]:
# loaddata
X, Y = loaddata('hw3_train.dat')
Xtest, Ytest = loaddata('hw3_test.dat')

In [16]:

def internal_node(node):
    if node == None:
        return 0
    if node.leftNode == None and node.rightNode == None:
        return 0
    l = 0; r = 0
    if node.leftNode != None:
        l = internal_node(node.leftNode)
    if node.rightNode != None:
        r = internal_node(node.rightNode)
    return 1 + l + r
node = dTree(X, Y)
print 'number of nodes on fully grown tree:', internal_node(node)

number of nodes on fully grown tree: 10


In [17]:
# Q14 and Q15
yin, ein = err_fun(X, Y, node)
yout, eout = err_fun(Xtest, Ytest, node)
print 'Ein:', ein, '\nEout:', eout

Ein: 0.0 
Eout: 0.126


In [18]:

# Q16, Q17, Q18
ein = 0; eout = 0; err = 0
for j in range(50):
    nodeArr = random_forest(X, Y, 300)
    l = len(nodeArr)
    yhat1 = np.zeros((Y.shape[0], l))
    yhat2 = np.zeros((Ytest.shape[0], l))
    for i in range(l):
        yhat1[:, i:i+1], _ = err_fun(X, Y, nodeArr[i])
        yhat2[:, i:i+1], _ = err_fun(Xtest, Ytest, nodeArr[i])
    #errg = float(np.sum(yhat1!=Y, 0))/Y.shape[0]
    errg = np.sum(yhat1!=Y, 0)/float(Y.shape[0])
    Yhat = np.sign(np.sum(yhat1, 1)).reshape(Y.shape)
    Ytesthat = np.sign(np.sum(yhat2, 1)).reshape(Ytest.shape)
    Yhat[Yhat==0] = 1; Ytesthat[Ytesthat==0] = 1
    ein += float(np.sum(Yhat!=Y))/Y.shape[0]
    eout += float(np.sum(Ytesthat!=Ytest))/Ytest.shape[0]
    err += float(np.sum(errg))/l 
print 'average eins for small gt:', err/50
print 'ein:', ein/50
print 'eout:', eout/50

average eins for small gt: 0.0522686666667
ein: 0.0
eout: 0.0752


In [19]:
# Q19 Q20
ein = 0; eout = 0
for j in range(50):
    nodeArr = random_forest_pruned(X, Y, 300)
    l = len(nodeArr)
    yhat1 = np.zeros((Y.shape[0],l))
    yhat2=np.zeros((Ytest.shape[0],l))
    for i in range(l):
        yhat1[:, i:i+1], _ = err_fun(X, Y, nodeArr[i])
        yhat2[:, i:i+1], _ = err_fun(Xtest, Ytest, nodeArr[i])
    Yhat = np.sign(np.sum(yhat1, 1)).reshape(Y.shape)
    Ytesthat = np.sign(np.sum(yhat2, 1)).reshape(Ytest.shape)
    Yhat[Yhat == 0] = 1
    Ytesthat[Ytesthat == 0] = 1
    ein += float(np.sum(Yhat != Y))/Y.shape[0]
    eout += float(np.sum(Ytesthat!=Ytest))/Ytest.shape[0]
print 'ein:', ein/50, '\neout:', eout/50

ein: 0.1132 
eout: 0.15262
